In [ ]:
import pandas as pd
import re
import pymorphy3
import nltk
from nltk.corpus import stopwords
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# Загрузка стоп-слов для русского языка (если еще не загружены)
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    """
    Предварительная обработка:
    - Приведение к нижнему регистру
    - Удаление пунктуации
    - Токенизация (разбивка по пробелам)
    - Удаление русских стоп-слов
    - Лемматизация
    """
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in russian_stopwords]
    return " ".join(tokens)

# Чтение данных из Excel файла
df = pd.read_excel("results\reviews.xlsx")

# --- ВЫВОД СПИСКА ДОСТУПНЫХ ЭМОЦИЙ ---

# Функция для извлечения уникальных эмоций из колонки "Эмоции"
def extract_unique_emotions(emotions_series):
    unique_emotions = set()
    for val in emotions_series.dropna():
        # Разбиваем по запятой, очищаем пробелы и приводим к нижнему регистру
        emotions = [e.strip().lower() for e in str(val).split(",") if e.strip()]
        unique_emotions.update(emotions)
    return sorted(list(unique_emotions))

unique_emotions = extract_unique_emotions(df["Эмоции"])

# Выводим список доступных эмоций с номерами
print("Доступные эмоции:")
for idx, emotion in enumerate(unique_emotions):
    print(f"{idx}: {emotion}")

# Запрашиваем у пользователя выбор эмоций по номерам
emotion_indices_str = input("Введите номера эмоций для фильтрации (через запятую) или оставьте пустым: ").strip()
if emotion_indices_str:
    try:
        # Извлекаем номера и получаем соответствующие эмоции
        selected_indices = [int(idx.strip()) for idx in emotion_indices_str.split(",") if idx.strip().isdigit()]
        selected_emotions = [unique_emotions[i] for i in selected_indices if i < len(unique_emotions)]
    except Exception as e:
        print("Ошибка при выборе эмоций. Фильтрация по эмоциям не будет применяться.")
        selected_emotions = []
else:
    selected_emotions = []

# --- ДОПОЛНИТЕЛЬНАЯ ФИЛЬТРАЦИЯ ПО КЛАССИФИКАЦИИ ---

# Запрашиваем номера классов для фильтрации (формат: 0,4,...)
classification_filter_str = input("Введите номера классов для фильтрации (через запятую, например: 0,4) или оставьте пустым: ").strip()
if classification_filter_str:
    try:
        filter_class_indices = [int(idx.strip()) for idx in classification_filter_str.split(",") if idx.strip().isdigit()]
    except ValueError:
        print("Ошибка в вводе номеров классов. Фильтрация по классам отключена.")
        filter_class_indices = []
else:
    filter_class_indices = []

# --- ДОПОЛНИТЕЛЬНАЯ ФИЛЬТРАЦИЯ ПО ОЦЕНКАМ ---

# Запрашиваем у пользователя конкретные оценки для фильтрации (например, "4,5")
ratings_filter_str = input("Введите оценки (0-5) через запятую для фильтрации (например: 4,5) или оставьте пустым: ").strip()
if ratings_filter_str:
    try:
        rating_filter_values = [int(r.strip()) for r in ratings_filter_str.split(",") if r.strip().isdigit()]
    except ValueError:
        print("Ошибка в вводе оценок. Фильтрация по оценкам отключена.")
        rating_filter_values = []
else:
    rating_filter_values = []

# Функция для проверки, содержит ли отзыв хотя бы одну из выбранных эмоций
def check_emotions(row_emotions, selected_emotions):
    if not selected_emotions:
        return True
    # Приводим строку с эмоциями к списку
    emotions = [e.strip().lower() for e in str(row_emotions).split(",") if e.strip()]
    return any(e in emotions for e in selected_emotions)

# Функция для проверки классификации
def check_classification(class_str, filter_class_indices):
    if not filter_class_indices:
        return True
    try:
        class_vals = [int(x) for x in class_str.split(",")]
    except Exception:
        return False
    return any((idx < len(class_vals)) and (class_vals[idx] == 1) for idx in filter_class_indices)

# Функция для проверки оценки
def check_rating(rating, rating_filter_values):
    if not rating_filter_values:
        return True
    return rating in rating_filter_values

# Применяем фильтрацию по эмоциям, классификации и оценкам
df_filtered = df[
    df["Эмоции"].apply(lambda x: check_emotions(x, selected_emotions)) &
    df["Классификация"].apply(lambda x: check_classification(x, filter_class_indices)) &
    df["Оценка"].apply(lambda x: check_rating(x, rating_filter_values))
]

print(f"\nОтобрано {len(df_filtered)} отзывов из исходных {len(df)} по заданным критериям.")

# --- ПРЕДОБРАБОТКА ТЕКСТОВ ---

# Предполагаем, что текст отзыва находится в столбце 'Отзыв'
df_filtered['processed_review'] = df_filtered['Отзыв'].apply(preprocess_text)
texts = df_filtered['processed_review'].tolist()

# --- ТЕМАТИЧЕСКОЕ МОДЕЛИРОВАНИЕ С BERTopic ---

# Используем модель для русского языка
embedder = SentenceTransformer("cointegrated/rubert-tiny2")

cluster_model = HDBSCAN(min_cluster_size=3, min_samples=5, metric='euclidean', cluster_selection_method='eom')

topic_model = BERTopic(
    embedding_model=embedder,
    hdbscan_model=cluster_model,
    language="russian",
    verbose=True
)

topics, probs = topic_model.fit_transform(texts)

print("\nИнформация по темам до редукции:")
print(topic_model.get_topic_info())

# Опционально: объединение схожих тем
# topic_model.reduce_topics(texts, nr_topics=50)
#print("\nИнформация по темам после редукции:")
#print(topic_model.get_topic_info())

# Визуализация распределения тем
fig = topic_model.visualize_topics()
fig.show()

# Вывод детальной информации для темы с идентификатором 0 (если такая тема есть)
if 0 in topic_model.get_topic_info()["Topic"].values:
    print("\nДетальная информация по теме 0:")
    print(topic_model.get_topic(0))
else:
    print("Тема 0 не найдена. Проверьте информацию по темам.")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\redka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Доступные эмоции:
0: admiration
1: amusement
2: anger
3: annoyance
4: approval
5: caring
6: confusion
7: curiosity
8: desire
9: disappointment
10: disapproval
11: disgust
12: embarrassment
13: excitement
14: fear
15: gratitude
16: grief
17: joy
18: love
19: nervousness
20: neutral
21: optimism
22: pride
23: realization
24: relief
25: remorse
26: sadness
27: surprise

Отобрано 5109 отзывов из исходных 5109 по заданным критериям.


2025-05-11 22:08:46,502 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/160 [00:00<?, ?it/s]

2025-05-11 22:08:57,182 - BERTopic - Embedding - Completed ✓
2025-05-11 22:08:57,182 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-11 22:08:59,040 - BERTopic - Dimensionality - Completed ✓
2025-05-11 22:08:59,041 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-11 22:08:59,216 - BERTopic - Cluster - Completed ✓
2025-05-11 22:08:59,219 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-05-11 22:08:59,706 - BERTopic - Representation - Completed ✓



Информация по темам до редукции:
     Topic  Count                                               Name  \
0       -1   2975                 -1_очень_спасибо_операции_операцию   
1        0    334   0_благодарность_здоровья_спасибо_профессионализм   
2        1    114        1_отделения_выразить_здоровья_благодарность   
3        2    108                         2_это_елисей_день_операции   
4        3     56                      3_деньги_другую_сказали_итоге   
..     ...    ...                                                ...   
158    157      3                 157_ульяновске_умеет_годом_дмитрий   
159    158      3                       158_02_хирург_надёжных_нечем   
160    159      3  159_аортокоронарному_высококвалифицированную_ш...   
161    160      3        160_ксение_переступили_ваильеву_проявленную   
162    161      3                               161_муж_ход_явное_сф   

                                        Representation  \
0    [очень, спасибо, операции, операцию, о


Детальная информация по теме 0:
[('благодарность', 0.012637684185290026), ('здоровья', 0.012349005556723464), ('спасибо', 0.012202925796131603), ('профессионализм', 0.011034104855782467), ('хочу', 0.010837815528101629), ('выразить', 0.01074458417750813), ('огромную', 0.009231499101368367), ('уважением', 0.009043040184433986), ('большое', 0.008924781923127309), ('отношение', 0.008911493028783862)]


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Загрузка стоп-слов для русского языка (если еще не загружены)
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    """
    Предварительная обработка:
    - Приведение к нижнему регистру
    - Удаление пунктуации
    - Токенизация (разбивка по пробелам)
    - Удаление русских стоп-слов
    """
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in russian_stopwords]
    return " ".join(tokens)

# Чтение данных из Excel файла
df = pd.read_excel("results\reviews.xlsx")

# --- ВЫВОД СПИСКА ДОСТУПНЫХ ЭМОЦИЙ --- 

# Функция для извлечения уникальных эмоций из колонки "Эмоции"
def extract_unique_emotions(emotions_series):
    unique_emotions = set()
    for val in emotions_series.dropna():
        # Разбиваем по запятой, очищаем пробелы и приводим к нижнему регистру
        emotions = [e.strip().lower() for e in str(val).split(",") if e.strip()]
        unique_emotions.update(emotions)
    return sorted(list(unique_emotions))

unique_emotions = extract_unique_emotions(df["Эмоции"])

# Выводим список доступных эмоций с номерами
print("Доступные эмоции:")
for idx, emotion in enumerate(unique_emotions):
    print(f"{idx}: {emotion}")

# Запрашиваем у пользователя выбор эмоций по номерам
emotion_indices_str = input("Введите номера эмоций для фильтрации (через запятую) или оставьте пустым: ").strip()
if emotion_indices_str:
    try:
        # Извлекаем номера и получаем соответствующие эмоции
        selected_indices = [int(idx.strip()) for idx in emotion_indices_str.split(",") if idx.strip().isdigit()]
        selected_emotions = [unique_emotions[i] for i in selected_indices if i < len(unique_emotions)]
    except Exception as e:
        print("Ошибка при выборе эмоций. Фильтрация по эмоциям не будет применяться.")
        selected_emotions = []
else:
    selected_emotions = []

# --- ДОПОЛНИТЕЛЬНАЯ ФИЛЬТРАЦИЯ ПО КЛАССИФИКАЦИИ ---

# Запрашиваем номера классов для фильтрации (формат: 0,4,...)
classification_filter_str = input("Введите номера классов для фильтрации (через запятую, например: 0,4) или оставьте пустым: ").strip()
if classification_filter_str:
    try:
        filter_class_indices = [int(idx.strip()) for idx in classification_filter_str.split(",") if idx.strip().isdigit()]
    except ValueError:
        print("Ошибка в вводе номеров классов. Фильтрация по классам отключена.")
        filter_class_indices = []
else:
    filter_class_indices = []

# Функция для проверки, содержит ли отзыв хотя бы одну из выбранных эмоций
def check_emotions(row_emotions, selected_emotions):
    if not selected_emotions:
        return True
    # Приводим строку с эмоциями в список
    emotions = [e.strip().lower() for e in str(row_emotions).split(",") if e.strip()]
    return any(e in emotions for e in selected_emotions)

# Функция для проверки классификации
def check_classification(class_str, filter_class_indices):
    if not filter_class_indices:
        return True
    try:
        class_vals = [int(x) for x in class_str.split(",")]
    except Exception:
        return False
    return any((idx < len(class_vals)) and (class_vals[idx] == 1) for idx in filter_class_indices)

# Применяем фильтрацию по эмоциям и классификации
df_filtered = df[
    df["Эмоции"].apply(lambda x: check_emotions(x, selected_emotions)) &
    df["Классификация"].apply(lambda x: check_classification(x, filter_class_indices))
]

print(f"\nОтобрано {len(df_filtered)} отзывов из исходных {len(df)} по заданным критериям.")

# --- ПРЕДОБРАБОТКА ТЕКСТОВ ---

# Предполагаем, что текст отзыва находится в столбце 'Отзыв'
df_filtered['processed_review'] = df_filtered['Отзыв'].apply(preprocess_text)
texts = df_filtered['processed_review'].tolist()

# --- ТЕМАТИЧЕСКОЕ МОДЕЛИРОВАНИЕ С BERTopic ---

from sentence_transformers import SentenceTransformer
# Используем модель для русского языка
embedder = SentenceTransformer("cointegrated/rubert-tiny")

from hdbscan import HDBSCAN
from umap import UMAP
cluster_model = HDBSCAN(min_cluster_size=4, min_samples=5, metric='euclidean', cluster_selection_method='eom')
umap_model = UMAP(n_components=5, n_neighbors=15, metric='cosine', random_state = 42)

from bertopic import BERTopic
topic_model = BERTopic(
    embedding_model=embedder,
    hdbscan_model=cluster_model,
    language="russian",
    umap_model=umap_model,
    verbose=True
)

topics, probs = topic_model.fit_transform(texts)

print("\nИнформация по темам до редукции:")
print(topic_model.get_topic_info())

# Опционально: объединение схожих тем
print("\nИнформация по темам после редукции:")
print(topic_model.get_topic_info())

# Визуализация распределения тем
fig = topic_model.visualize_topics()
fig.show()

# Вывод детальной информации для темы с идентификатором 0 (если такая тема есть)
if 0 in topic_model.get_topic_info()["Topic"].values:
    print("\nДетальная информация по теме 0:")
    print(topic_model.get_topic(0))
else:
    print("Тема 0 не найдена. Проверьте информацию по темам.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\redka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Доступные эмоции:
0: admiration
1: amusement
2: anger
3: annoyance
4: approval
5: caring
6: confusion
7: curiosity
8: desire
9: disappointment
10: disapproval
11: disgust
12: embarrassment
13: excitement
14: fear
15: gratitude
16: grief
17: joy
18: love
19: nervousness
20: neutral
21: optimism
22: pride
23: realization
24: relief
25: remorse
26: sadness
27: surprise

Отобрано 5109 отзывов из исходных 5109 по заданным критериям.


No sentence-transformers model found with name cointegrated/rubert-tiny. Creating a new one with mean pooling.
2025-05-16 19:50:48,884 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/160 [00:00<?, ?it/s]

2025-05-16 19:50:53,800 - BERTopic - Embedding - Completed ✓
2025-05-16 19:50:53,800 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-16 19:51:20,518 - BERTopic - Dimensionality - Completed ✓
2025-05-16 19:51:20,527 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-16 19:51:20,747 - BERTopic - Cluster - Completed ✓
2025-05-16 19:51:20,761 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-05-16 19:51:21,201 - BERTopic - Representation - Completed ✓



Информация по темам до редукции:
     Topic  Count                                              Name  \
0       -1   3112                -1_очень_спасибо_операции_операцию   
1        0    313              0_клиника_специалисты_персонал_врачи   
2        1    119  1_благодарность_выразить_спасибо_профессионализм   
3        2    112             2_врач_рекомендации_подробно_объяснил   
4        3     82                        3_это_день_елисей_операции   
..     ...    ...                                               ...   
117    116      4                    116_иван_извлечь_однажды_ночью   
118    117      4        117_220922_перинеопластика_вязилова_ничуть   
119    118      4             118_благодарю_валерьевна_юлия_выделяю   
120    119      4  119_резекции_александровичу_хлопоты_эстетические   
121    120      4                       120_отчаянно_1800_тк_января   

                                        Representation  \
0    [очень, спасибо, операции, операцию, операция,... 


Детальная информация по теме 0:
[('клиника', 0.059982852120772534), ('специалисты', 0.059325429165852345), ('персонал', 0.04622746614228585), ('врачи', 0.04053289940948367), ('хорошая', 0.03998171221080939), ('вежливый', 0.03456485191434869), ('отличные', 0.034104717780786144), ('отличная', 0.028691301122084657), ('цены', 0.02708716970272803), ('хорошие', 0.026694140572836245)]


In [8]:
top_topics = [2, 4, 7, 8, 9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28,
           29, 31, 32, 34, 35, 37, 42, 43, 44, 45, 46, 47, 51, 55, 56, 57, 61, 62, 63,
           65, 72, 73, 78, 89, 92, 93, 94, 96, 98]

In [ ]:
short_topics = [2, 4, 7, 8, 9, 10, 13, 14, 15, 16, 18, 19]
fig1 = topic_model.visualize_barchart(custom_labels=True, width= 380, n_words=10, height= 300,
                                    title='Распределение слов по темам')
fig1.write_html(r"results/topics.html")
fig2 = topic_model.visualize_barchart(top_topics, custom_labels=True, width= 390,
                                    title='Распределение слов по темам')
fig2.write_html(r"results/topics_full.html")
fig2

In [14]:
import pandas as pd

# Получаем сводную информацию по темам
topic_info = topic_model.get_topic_info()
# Получаем репрезентативные документы для всех тем
rep_docs = topic_model.get_representative_docs()

# Создадим список для сохранения результатов
topics_data = []

# Проходим по темам из topic_info
for topic in topic_info["Topic"]:
    # Пропускаем шум (метка -1)
    if topic == -1:
        continue

    # Извлекаем ключевые слова для данной темы
    # Функция get_topic(topic) возвращает список кортежей (слово, вес)
    keywords_tuples = topic_model.get_topic(topic)
    if keywords_tuples is None:  # если тема пуста, пропускаем
        continue
    # Оставляем только слова и объединяем через запятую
    keywords = ", ".join([word for word, weight in keywords_tuples])
    
    # Получаем до 10 репрезентативных отзывов для темы
    # Метод get_representative_docs() возвращает словарь:
    # ключ – номер темы, значение – список списков отзывов;
    # Берём первые 10 отзывов, если их много
    representative_reviews = rep_docs.get(topic, [])
    # Иногда репрезентативные документы могут быть возвращены списком списков;
    # преобразуем их в один список, если это необходимо
    if representative_reviews and isinstance(representative_reviews[0], list):
        representative_reviews = [item for sublist in representative_reviews for item in sublist]
    representative_reviews = representative_reviews[:2]
    
    topics_data.append({
        "topic": topic,
        "keywords": keywords,
        "representative_reviews": representative_reviews
    })

# Преобразуем результаты в DataFrame для наглядности
df_topics = pd.DataFrame(topics_data)

# Сохраняем результаты в CSV файл. Ключевые слова будут записаны как строка,
# а репрезентативные отзывы – как строковое представление списка.
df_topics.to_excel("topics_analysis_new.xlsx", index=False)

# Выведем результаты на экран
print("Сохранены ключевые слова и репрезентативные отзывы для каждой темы:")
print(df_topics)


Сохранены ключевые слова и репрезентативные отзывы для каждой темы:
     topic                                           keywords  \
0        0  клиника, специалисты, персонал, врачи, хорошая...   
1        1  благодарность, выразить, спасибо, профессионал...   
2        2  врач, рекомендации, подробно, объяснил, клиник...   
3        3  это, день, елисей, операции, александрович, пр...   
4        4  руки, спасибо, железы, золотые, щитовидной, ог...   
..     ...                                                ...   
116    116  иван, извлечь, однажды, ночью, беспокоить, вла...   
117    117  220922, перинеопластика, вязилова, ничуть, сос...   
118    118  благодарю, валерьевна, юлия, выделяю, чистотап...   
119    119  резекции, александровичу, хлопоты, эстетически...   
120    120  отчаянно, 1800, тк, января, успокоит, прямой, ...   

                                representative_reviews  
0    [хорошее обслуживание грамотные специалисты ве...  
1    [хочу выразить огромную благодар

In [3]:
import pandas as pd
import re
import pymorphy3
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

# Загрузка стоп-слов для русского языка (если еще не загружены)
nltk.download("stopwords")
russian_stopwords = set(stopwords.words("russian"))

# Инициализация лемматизатора
morph = pymorphy3.MorphAnalyzer()

def preprocess_text(text):
    """
    Предварительная обработка:
    - Приведение к нижнему регистру
    - Удаление пунктуации
    - Токенизация (разбивка по пробелам)
    - Удаление русских стоп-слов
    - Лемматизация
    """
    # lowercase
    text = text.lower()
    # remove punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    # tokenize
    tokens = text.split()
    # remove stopwords
    tokens = [tok for tok in tokens if tok not in russian_stopwords]
    # lemmatize
    lemmas = []
    for tok in tokens:
        p = morph.parse(tok)
        if p:
            lemmas.append(p[0].normal_form)
    return " ".join(lemmas)

# Загрузка данных
# Предполагается, что файл содержит столбцы: 'Отзыв', 'Оценка', 'Эмоции', 'Классификация'
df = pd.read_excel("Продокторов_полный.xlsx")

# Фильтрация: оставляем только негативные отзывы (например, оценки <= 2)
df_negative = df[df['Оценка'].isin([0, 1, 2, 3])].copy()
print(f"Всего негативных отзывов: {len(df_negative)}")

# Препроцессинг текстов
df_negative['processed'] = df_negative['Отзыв'].astype(str).apply(preprocess_text)
texts = df_negative['processed'].tolist()

# Параметры n-грамм
ngram_min = 1  # минимальное значение n
ngram_max = 3  # максимальное значение n

top_n = 1000    # количество самых частых n-грамм для вывода

# Создаем мешок слов с n-граммами
vectorizer = CountVectorizer(ngram_range=(ngram_min, ngram_max), 
                             lowercase=False,  # уже понижено в preprocess_text
                             analyzer='word',
                             token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(texts)

# Суммируем частоты n-грамм
freq = X.sum(axis=0).A1
terms = vectorizer.get_feature_names_out()

# Создаем датафрейм с частотами
df_freq = pd.DataFrame({'ngram': terms, 'count': freq})
# Сортировка по убыванию
df_top = df_freq.sort_values('count', ascending=False).head(top_n)

# Вывод результата
print(f"Топ {top_n} наиболее частых n-грамм в негативных отзывах:")
for idx, row in df_top.iterrows():
    print(f"{row['ngram']}: {row['count']}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\redka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Всего негативных отзывов: 199
Топ 1000 наиболее частых n-грамм в негативных отзывах:
клиника: 209
врач: 200
это: 116
операция: 112
сказать: 100
приём: 90
очень: 89
который: 84
анализ: 74
время: 70
такой: 64
свой: 62
делать: 58
мой: 55
процедура: 54
другой: 52
2: 48
день: 46
доктор: 46
человек: 46
деньга: 44
хороший: 43
пациент: 42
год: 41
консультация: 40
сам: 40
сделать: 39
специалист: 39
результат: 39
проблема: 38
нужно: 38
место: 36
всё: 36
мочь: 36
вопрос: 36
просто: 35
минута: 35
регистратура: 33
запись: 33
лечение: 32
назначить: 32
итог: 32
дать: 32
пройти: 31
прийти: 31
знать: 30
стать: 30
какой: 30
хотеть: 29
цена: 29
быть: 28
3: 28
получить: 27
услуга: 27
один: 27
приехать: 26
первый: 26
администратор: 26
записаться: 25
месяц: 25
так: 25
неделя: 25
отношение: 25
персонал: 24
ждать: 24
клиент: 23
никто: 23
вообще: 23
работать: 23
5: 22
прийтись: 22
весь: 22
отделение: 22
должный: 22
хотя: 21
предложить: 21
глаз: 21
попасть: 21
кабинет: 21
работа: 21
самый: 20
помочь: 20
хирург: